In [1]:
using JuMP, JuMPeR, Gurobi, Random, Distributions, LinearAlgebra, Plots, CSV, DataFrames, Parsers

ArgumentError: [91mArgumentError: Package JuMP not found in current path:[39m
[91m- Run `import Pkg; Pkg.add("JuMP")` to install the JuMP package.[39m


# Lecture 7 Notes

## Nominal Model - Known returns

### Parameters

In [58]:
# Total number of assets; Number of risky assets = M-1
M = 20 

# Number of trading periods
N = 5

# brokerage fees
csell = 0.03
cbuy = 0.02

# returns by asset and period
#  riskless asset
r₀ = ones(N) 
rₘ = rand(N,M)

5×20 Array{Float64,2}:
 0.785203  0.819186   0.160995   0.13234   …  0.844656  0.411823   0.28412  
 0.366932  0.9452     0.762979   0.309001     0.550543  0.249389   0.769855 
 0.368773  0.0682555  0.3565     0.864989     0.517231  0.0060172  0.416996 
 0.436183  0.202712   0.955486   0.178517     0.458126  0.619826   0.0392332
 0.509694  0.204766   0.0140818  0.743806     0.398757  0.552265   0.581627 

### Formulation

In [59]:
mod = Model(solver = GurobiSolver())

##### Decision Variables #####

# x, number of dollar holdings at the beginning of time period t on asset m
@variable(mod, x[1:N, 0:M] >= 0)

# u, dollar sells on asset m at time t
@variable(mod, u[1:N, 0:M] >= 0)

# v, dollar buys on asset m at time t
@variable(mod, v[1:N, 0:M] >= 0)

##### Constraints #####

# buy and sell constraints
@constraint(mod, [t=2:N,m=1:M], x[t,m] == (1+rₘ[t-1,m])*(x[t-1,m] - u[t-1,m] + v[t-1,m]))
@constraint(mod, [t=2:N], x[t,0] == (1+r₀[t-1])*(x[t-1,0] + sum((1-csell)*u[t-1,m] for m = 1:M) - sum((1-cbuy)*v[t-1,m] for m = 1:M)))

## NEED A CONSTRAINT ON THE AMOUNT OF CASH AVAILABLE AT T=0
@constraint(mod, sum(x[1,m] for m in 0:M) <= 1)

##### Objective Function #####

# maximize expected final wealth
@objective(mod, Max, sum(x[N,m] for m in 0:M))

solve(mod)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 85 rows, 315 columns and 509 nonzeros
Model fingerprint: 0x37a747b7
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 19 rows and 107 columns
Presolve time: 0.00s
Presolved: 66 rows, 208 columns, 414 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   1.052547e+30   3.000000e+00      0s
      73    1.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 73 iterations and 0.00 seconds
Optimal objective  1.600000000e+01


:Optimal

## Robust Model - Reformulation

### Parameters

In [55]:
# Cumulative Returns for at time 0
R₀ = ones(M)

#Cumulative Returns at time t for asset m
Rₜ = []
for t in 1:N:
    
        


4-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0

In [ ]:
rmod = Model(solver = GurobiSolver())

##### Decision Variables #####

# x, number of dollar holdings at the beginning of time period t on asset m
@variable(rmod, x[1:N, 0:M] >= 0)

# u, dollar sells on asset m at time t
@variable(rmod, u[1:N, 0:M] >= 0)

# v, dollar buys on asset m at time t
@variable(rmod, v[1:N, 0:M] >= 0)

# Reformulation Variables


##### Constraints #####

# buy and sell constraints
@constraint(rmod, [t=2:N,m=1:M], x[t,m] == (1+rₘ[t-1,m])*(x[t-1,m] - u[t-1,m] + v[t-1,m]))
@constraint(rmod, [t=2:N], x[t,0] == (1+r₀[t-1])*(x[t-1,0] + sum((1-csell)*u[t-1,m] for m = 1:M) - sum((1-cbuy)*v[t-1,m] for m = 1:M)))

########### NEED A CONSTRAINT ON THE AMOUNT OF CASH AVAILABLE AT T=0
@constraint(rmod, sum(x[1,m] for m in 0:M) <= 1)




##### Objective Function #####

# maximize expected final wealth
@objective(rmod, Max, sum(x[N,m] for m in 0:M))

solve(rmod)